In [16]:
# Please execute/shift-return this cell everytime you run the notebook.  Don't edit it. 
%load_ext autoreload
%autoreload 2
from notebook import * 

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# Virtual Memory

What if we run the following program on a machine with 32GB memory?

In [17]:
render_code("memory_allocation.c")

// memory_allocation.c:1-38 (38 lines)
#define _GNU_SOURCE  
#include <unistd.h>
#include <stdio.h>
#include <stdlib.h>
#include <assert.h>
#include <sched.h>
#include <sys/syscall.h>
#include <time.h>

//#define dim 32768
//#define dim 49152
int main(int argc, char *argv[])
{
    int i,j;
    double *a;
    double sum=0, average, value;
    int dim=32768;
    if(argc < 2)
    {
        fprintf(stderr, "Usage: %s dimension\n",argv[0]);
        exit(1);
    }
    dim = atoi(argv[1]);
    value = atof(argv[2]);
    a = (double *)malloc(sizeof(double)*dim*dim);
//    for(i = 0 ; i < dim; i++)
//        a[i] = (double *)malloc(sizeof(double)*dim);
    for(i = 0 ; i < dim; i++)
        for(j = 0 ; j < dim; j++)
            a[i*dim+j] = value*i/j;
    for(i = 0 ; i < dim; i++)
        for(j = 0 ; j < dim; j++)
            sum+=a[i*dim+j];
    average = sum/(dim*dim);
    fprintf(stderr,"average: %lf\n",average);
    free(a);
    return 0;
}

In [19]:
! ssh htseng@snorlax "free -m ; cd courses/CSE142/demo/virtual_memory/; make clean ./memory_allocation; ./memory_allocation 48000 3.14"

              total        used        free      shared  buff/cache   available
Mem:          15756         795       10073          10        4887       14613
Swap:         67583           0       67583
rm -f virtualization memory_allocation 
gcc -O3 memory_allocation.c -o memory_allocation
average: -20.631999


What's the output of the following code if we run 4 instances at the same time?

In [20]:
render_code("virtualization.c")

// virtualization.c:1-28 (28 lines)
#define _GNU_SOURCE  
#include <unistd.h>
#include <stdio.h>
#include <stdlib.h>
#include <assert.h>
#include <sched.h>
#include <sys/syscall.h>
#include <time.h>

double a = 0;

int main(int argc, char *argv[])
{
    int i, number_of_total_processes=4;
    if(argc < 2)
    {
        fprintf(stderr, "Usage: %s number_of_processes\n",argv[0]);
        exit(1);
    }
    number_of_total_processes = atoi(argv[1]);
    for(i = 0; i< number_of_total_processes-1 && fork(); i++);
    srand((int)time(NULL)+(int)getpid());
    a = rand();
    fprintf(stderr, "\nProcess %d: Value of a is %lf and address of a is %p\n",(int)getpid(), a, &a);
    sleep(10);
    fprintf(stderr, "\nProcess %d: Value of a is %lf and address of a is %p\n",(int)getpid(), a, &a);
    return 0;
}

In [ ]:
! make clean virtualization ; ./virtualization 4

In [30]:
! ssh htseng@pikachu "cpuid -1"

CPU:
   cache and TLB information (2):
   0x00000002 0x00: eax=0x00feff01 ebx=0x00000000 ecx=0x00000000 edx=0x00000000


In [35]:
! ssh htseng@blissey "cpuid -1"

CPU:
   vendor_id = "AuthenticAMD"
   version information (1/eax):
      processor type  = primary processor (0)
      family          = 0xf (15)
      model           = 0x1 (1)
      stepping id     = 0x2 (2)
      extended family = 0xa (10)
      extended model  = 0x2 (2)
      (family synth)  = 0x19 (25)
      (model synth)   = 0x21 (33)
   miscellaneous (1/ebx):
      process local APIC physical ID = 0x9 (9)
      maximum IDs for CPUs in pkg    = 0x10 (16)
      CLFLUSH line size              = 0x8 (8)
      brand index                    = 0x0 (0)
   brand id = 0x00 (0): unknown
   feature information (1/edx):
      x87 FPU on chip                        = true
      VME: virtual-8086 mode enhancement     = true
      DE: debugging extensions               = true
      PSE: page size extensions              = true
      TSC: time stamp counter                = true
      RDMSR and WRMSR support                = true
      PAE: physical address extensions       = true
      MCE: ma

In [34]:
! ssh htseng@snorlax "cpuid -1"

CPU:
   vendor_id = "GenuineIntel"
   version information (1/eax):
      processor type  = primary processor (0)
      family          = 0x6 (6)
      model           = 0x7 (7)
      stepping id     = 0x5 (5)
      extended family = 0x0 (0)
      extended model  = 0x9 (9)
      (family synth)  = 0x6 (6)
      (model synth)   = 0x97 (151)
      (simple synth)  = Intel (unknown model)
   miscellaneous (1/ebx):
      process local APIC physical ID = 0x0 (0)
      cpu count                      = 0x10 (16)
      CLFLUSH line size              = 0x8 (8)
      brand index                    = 0x0 (0)
   brand id = 0x00 (0): unknown
   feature information (1/edx):
      x87 FPU on chip                        = true
      VME: virtual-8086 mode enhancement     = true
      DE: debugging extensions               = true
      PSE: page size extensions              = true
      TSC: time stamp counter                = true
      RDMSR and WRMSR support                = true
      PAE: physical ad